In [1]:
import numpy as np
import math

In [2]:
currenies = {0: "USD",1:"EUR",2:"GBP",3:"JPY",4:"CHF",5:"CAD",6:"AUD",7:"RUB"}
adj = [
    [1, 0.8651, 0.7340, 111.77, 0.9290, 1.2538, 1.3692, 72.0144],
    [1.1559, 1, 0.8483, 129.19, 1.0740, 1.4492, 1.5826, 83.2316],
    [1.3626, 1.1788, 1, 152.29, 1.2657, 1.7083, 1.8656, 98.124],
    [0.0089, 0.0077, 0.00657, 1, 0.0083, 0.01122, 0.01225, 0.6443],
    [1.0765, 0.9312, 0.7901, 120.29, 1, 1.3495, 1.4739, 77.52],
    [0.7976, 0.6901, 0.5854, 89.15, 0.7411, 1, 1.0921, 57.43],
    [0.7304, 0.6319, 0.5360, 81.64, 0.6784, 0.9157, 1, 52.60],
    [0.01389, 0.01201, 0.01019, 1.5520, 0.01290, 0.01741, 0.01901,1]
      ]
adj_log = -np.log(adj)
print("-adj_log\n",np.round(adj_log,2))

-adj_log
 [[-0.    0.14  0.31 -4.72  0.07 -0.23 -0.31 -4.28]
 [-0.14 -0.    0.16 -4.86 -0.07 -0.37 -0.46 -4.42]
 [-0.31 -0.16 -0.   -5.03 -0.24 -0.54 -0.62 -4.59]
 [ 4.72  4.87  5.03 -0.    4.79  4.49  4.4   0.44]
 [-0.07  0.07  0.24 -4.79 -0.   -0.3  -0.39 -4.35]
 [ 0.23  0.37  0.54 -4.49  0.3  -0.   -0.09 -4.05]
 [ 0.31  0.46  0.62 -4.4   0.39  0.09 -0.   -3.96]
 [ 4.28  4.42  4.59 -0.44  4.35  4.05  3.96 -0.  ]]


In [3]:
def bellman_ford(adj, src):

    # Step 1: Initialize distances from origin to all
    # dist = [np.inf] * len(adj)
    dist = [np.inf] * len(adj)
    pred = [-1] * len(adj)
    dist[src] = 0
    
    # Step 2: Relaxation for all edges |V|-1 times.
    for _ in range(len(dist)-1):
        for u in range(len(dist)):
            for v in range(len(dist)):
                if(u != v and not (v in pred) and dist[v] > dist[u] + adj[u][v]):
                    dist[v] = dist[u] + adj[u][v]
                    pred[v] = u
        # print("dist ", np.round(dist,2))
        # print("pred ",pred)
    if -1 in pred: 
        ver = np.array(range(8)).sum() - np.array(pred).sum() - 1
        pred[pred.index(-1)] = ver
    # print("\n")
    # print("dist ", np.round(dist,2))
    # print("pred ",pred)
    
    # Step 3: Check for negative-weight cycles
    for u in range(len(dist)):
        for v in range(len(dist)):
            if u != v and dist[u] != np.inf and dist[u] + adj[u][v] < dist[v]:
                # print("Negative weight cycle!")
                return False, dist, pred
    
    return True, dist, pred # There is no negative cycle

In [4]:
def dijkstra(adj, src):
    passed = [src]
    nopassed = [x for x in range(len(adj)) if x != src]
    dist = adj[src]
    pred = [src] * len(adj)
    
    while len(nopassed):
        idx = nopassed[0]
        for i in nopassed:
            if dist[i] < dist[idx]:
                idx = i
        
        nopassed.remove(idx)
        passed.append(idx)
        
        for i in nopassed:
            if dist[idx] + adj[idx][i] < dist[i]:
                dist[i] = dist[idx] + adj[idx][i]
                pred[i] = idx 
                
        # print("nopassed",nopassed)
        # print("passed",passed)
        # print("dist", np.round(dist,2))
        # print("pred", pred)
        # print("\n")
    return dist, pred

In [5]:
def resursion(idx, pred, N):
    if N == 0:
        return str(idx)
    return resursion(pred[idx]) + " -> " + str(idx)

In [6]:
def find_minus_cycle(adj, src):
    minus_cycle, dist, pred = bellman_ford(adj, src)
    if minus_cycle == False:
        # find the cycle:
        cycle = [[]]
        idx = 0
        for u in range(len(dist)):
            for v in range(len(dist)):
                if u != v and dist[u] + adj[u][v] < dist[v]:
                    # relaxataion : v -> back |V| steps
                    for i in range(len(adj)+1):
                        if i != len(adj):
                            cycle[idx].append(v)
                            v = pred[v]          
                    idx = idx + 1
                    cycle.append([])
        cycle.pop()
        
        # find-min
        min_weight = np.inf
        min_path = []
        for path in cycle:
            sum_weight = 0
            for k in path:
                sum_weight += adj[pred[k]][k]
            if sum_weight < min_weight: 
                min_weight = sum_weight
                min_path = path
        
        return min_path[::-1], min_weight

In [7]:
def list_move_left(A,a):
    for i in range(a):
        A.insert(len(A),A[0])
        A.remove(A[0])
    return A

In [8]:
def path_original(adj):
    adj_minus_log = -np.log(adj)
    abrita_array = []
    for i in range(8):
        min_path, min_weight = find_minus_cycle(adj_minus_log, i)
        product = 1
        for k in range(7):
            product *= adj[min_path[k]][min_path[k+1]]
        product *= adj[min_path[7]][min_path[0]]
        
        abrita_array.append(product)
        min_path = list_move_left(min_path, min_path.index(i))
        min_path.append(i)
        
        print("from "+ str(i) + " : " + str(min_path))
        print(product)
        
        print("\n")

In [9]:
path_original(adj)

from 0 : [0, 2, 3, 5, 1, 4, 6, 7, 0]
1.0009966430467783


from 1 : [1, 3, 5, 6, 7, 0, 2, 4, 1]
1.0005572875046422


from 2 : [2, 3, 5, 6, 7, 0, 1, 4, 2]
1.0008460874930094


from 3 : [3, 5, 6, 7, 0, 1, 4, 2, 3]
1.0008460874930094


from 4 : [4, 6, 7, 0, 2, 3, 5, 1, 4]
1.0009966430467783


from 5 : [5, 6, 7, 5, 6, 7, 6, 7, 5]
1.0001420928824434


from 6 : [6, 7, 0, 2, 3, 5, 4, 1, 6]
1.000781423451686


from 7 : [7, 0, 2, 3, 5, 1, 4, 6, 7]
1.0009966430467785


